In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import re
from nltk import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
import nltk
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('wordnet')
import dl_translate as dlt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\linco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\linco\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
  
tokenizerR = AutoTokenizer.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis")

modelR = TFAutoModelForSequenceClassification.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis", from_pt = True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [7]:
classifier = pipeline("sentiment-analysis")
classifier("I loved Star Wars so much!")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


[{'label': 'POSITIVE', 'score': 0.999840259552002}]

In [8]:
tokenizer = AutoTokenizer.from_pretrained("unicamp-dl/translation-pt-en-t5")
model = AutoModelForSeq2SeqLM.from_pretrained("unicamp-dl/translation-pt-en-t5")
pten_pipeline = pipeline('text2text-generation', model=model, tokenizer=tokenizer)
mt = dlt.TranslationModel()  # Slow when you load it for the first time

In [9]:
example_tweet = "The NEET exams show our Govt in a poor light: unresponsiveness to genuine concerns; admit cards not delivered to aspirants in time; failure to provide centres in towns they reside, thus requiring unnecessary & risky travels. What a disgrace to treat our #Covid warriors like this!"
#this tweet resides on Twitter with an identifier-1435793872588738560
    
input = tokenizerR.encode(example_tweet, return_tensors="tf")
output = modelR.predict(input)[0]
prediction = tf.nn.softmax(output, axis=1).numpy()
sentiment = np.argmax(prediction)
    
print(prediction)
print(sentiment)

[[0.97267216 0.02368472 0.00364307]]
0


In [12]:
df_portuguese = pd.read_csv("datasets/export_TweetSentBR.csv")
df_english = pd.read_csv("datasets/export_TweetSentEnglish.csv")
dataset = pd.read_csv("./datasetcompleto.csv")

In [6]:
df_remove = df_portuguese[df_portuguese['sentiment'] == '-']
df_portuguese = df_portuguese.drop(df_remove.index)
df_portuguese = df_portuguese.reset_index()
df_portuguese = df_portuguese.drop(columns=['index'])
df_portuguese['sentiment'] = df_portuguese['sentiment'].apply(lambda x: int(x))
Tweet = df_portuguese['text']
polarity = np.asarray(df_portuguese['sentiment'])

In [15]:
dataset.drop(columns = ['Unnamed: 0'], inplace = True)

KeyError: "['Unnamed: 0'] not found in axis"

In [22]:
# Tweets_in_English = pd.DataFrame( data = {'phrases': Tweets_in_Portuguese['frases'].apply(translater_phrases), 'classification': polarity})

In [16]:
# Tweets_in_Portuguese = pd.DataFrame( data = {'frases': Tweet ,'classificacao' : polarity})

In [19]:
df_portuguese['text'] = df_portuguese['text'].apply(remove_user)
df_portuguese.drop(columns = ['id', 'id_twitter'], inplace = True)

In [20]:
df_english.drop(columns = ['Unnamed: 0'], inplace = True)

In [21]:
df_english

,phrases,classification
0,The #encontro Program was showing a family tha...,1
1,I loved them.,1
2,CATRA launching its new song PPK CHORA no k,1
3,#MasterChefBR,0
4,"I'm handled with this guy.... how mu""",-1
...,...,...
11528,"I'm already here ready for #MasterChefBR but""",-1
11529,"One thing I don't have the courage is this""",-1
11530,#By You are beautiful #ByYou are beauti,1
11531,"What pride of you,! #I meet you",1


In [22]:
df_portuguese

,text,sentiment
0,Que coisa linda! O Programa #encontro estava m...,1
1,"Por mais #Encontro com as Irmãs Galvão, adorei...",1
2,Mr. CATRA lançando sua nova música PPK CHORA ...,1
3,quem viu aquela lutadora modela barbuda tatuad...,0
4,Tô passada com esse cara.... quanta merda pode...,-1
...,...,...
11565,eu ja to aqui pronto pro #MasterChefBR mas ain...,-1
11566,MALUCO! Uma coisa que eu não tenho coragem é e...,-1
11567,#MaisVoce está linda,1
11568,"Que orgulho de ti, ! #Encontro",1


In [9]:
def pre_X(frases):
    lista = []
    
    for frase in frases:
        lista.append(frase)
        
    return lista

def pre_Y(number):
    lista = []
    
    for numb in number:
        lista.append(numb)
    
    return lista


In [17]:
def set_array(frases):
    
    vocab = []
    palavras = []
    for frase in frases:
        
        text_array = remove_user(frase)
        text_array = Tokenize(text_array)
        text_array = text_array.split(' ')
        for i in range(len(text_array)):
            vocab.append(text_array[i])
    
    
        
    return vocab

def Tokenize(f):     ## Pre-processando a frase
    
    ## Colocando em minusculo
    ## Retirando a pontuaçao
    ## Retirando as StopWords
    
    f = f.lower().replace('\n', '').replace('-','').replace('#','').replace('.','').replace(',','').replace('!','').replace('r\n','').replace('  ','')
    token = RegexpTokenizer(r"\w+")
    f = token.tokenize(f)
    
    stop_words = set(stopwords.words('portuguese'))
    
    new_word = [word for word in f if not word in stop_words]
    
    return ' '.join(new_word)

def remove_user(frase):

    return re.sub('@\w+','',frase)

def translater_phrases(frase):
    translater = pten_pipeline(frase)
    translater = str(translater).strip('[{}]')
    translater = translater[19:]
    translater = translater.strip("''")
    
    return translater

def translater_frases(frase):
    text_hi = frase
    translater = mt.translate(text_hi, source=dlt.lang.PORTUGUESE, target=dlt.lang.ENGLISH)
    
    return translater

def new_classifier(frase):
    input = tokenizerR.encode(frase, return_tensors="tf")
    output = modelR.predict(input)[0]
    prediction = tf.nn.softmax(output, axis=1).numpy()
    sentiment = np.argmax(prediction)
    
    return sentiment 

def classifierD(frase):    
    return classifier(frase) 


In [21]:
Tweet = Tweet.apply(remove_user)
# Tweet_preprocessed = Tweet.apply(Tokenize)

# count_vect = CountVectorizer()
# X_train = count_vect.fit_transform(Tweet_preprocessed)

# tfidf_transformer = TfidfTransformer()
# X_train_transform = tfidf_transformer.fit_transform(X_train) # Aplicando o TF-IDF


# X_train, X_test, Y_train, Y_test = train_test_split(X_train_transform, polarity, test_size=0.3)

NameError: name 'Tweet' is not defined

In [14]:
Tweets_in_English = pd.DataFrame(data = {'tweets_original': df_portuguese['text'][:500], 'tweets_translater_unicamp': df_english['phrases'][:500],'tweets_translater_face': df_portuguese['text'][:500].apply(translater_frases)})

In [22]:
Tweets_in_English

,tweets_original,tweets_translater_unicamp,tweets_translater_face
0,Que coisa linda! O Programa #encontro estava m...,The #encontro Program was showing a family tha...,What a beautiful thing! The #touch program was...
1,"Por mais #Encontro com as Irmãs Galvão, adorei...",I loved them.,"For more I met with the Galvan Sisters, I love..."
2,Mr. CATRA lançando sua nova música PPK CHORA ...,CATRA launching its new song PPK CHORA no k,Mr. CATRA releases his new song PPK CHORA on k...
3,quem viu aquela lutadora modela barbuda tatuad...,#MasterChefBR,Who saw that fighter model tattooed? #MasterCh...
4,Tô passada com esse cara.... quanta merda pode...,"I'm handled with this guy.... how mu""",How much shit can come out of someone’s mouth ...
...,...,...,...
495,#VideoShowAoVivo Kkkk ahhh eu morro com esse V...,I love this guy!,#VideoShowAoLive Kkkk ahhh I die with this Sum...
496,"Até o Padre Fábio de Melo cantando Trem Bala, ...","Even Father Fábio de Melo singing Trem Bala, i...",Even the Fabius Father of Melo singing Trem Ba...
497,#MaisVocê minha irmã calça 34 e compra sapato ...,Never had difficulty in finding this number of...,You my sister shoes 34 and buy shoes in any st...
498,Iris tão fofa e linda 😻😻💙 #EDeCasa,Iris so fool and beautiful Iris #EDeCasa,Iris so foolish and beautiful 🏻 #EDeCasa


In [16]:
dataset

,tweets_original,tweets_translater_unicamp,tweets_translater_face
0,Que coisa linda! O Programa #encontro estava m...,The #encontro Program was showing a family tha...,What a beautiful thing! The #touch program was...
1,"Por mais #Encontro com as Irmãs Galvão, adorei...",I loved them.,"For more I met with the Galvan Sisters, I love..."
2,Mr. CATRA lançando sua nova música PPK CHORA ...,CATRA launching its new song PPK CHORA no k,Mr. CATRA releases his new song PPK CHORA on k...
3,quem viu aquela lutadora modela barbuda tatuad...,#MasterChefBR,Who saw that fighter model tattooed? #MasterCh...
4,Tô passada com esse cara.... quanta merda pode...,"I'm handled with this guy.... how mu""",How much shit can come out of someone’s mouth ...
...,...,...,...
495,#VideoShowAoVivo Kkkk ahhh eu morro com esse V...,I love this guy!,#VideoShowAoLive Kkkk ahhh I die with this Sum...
496,"Até o Padre Fábio de Melo cantando Trem Bala, ...","Even Father Fábio de Melo singing Trem Bala, i...",Even the Fabius Father of Melo singing Trem Ba...
497,#MaisVocê minha irmã calça 34 e compra sapato ...,Never had difficulty in finding this number of...,You my sister shoes 34 and buy shoes in any st...
498,Iris tão fofa e linda 😻😻💙 #EDeCasa,Iris so fool and beautiful Iris #EDeCasa,Iris so foolish and beautiful 🏻 #EDeCasa


In [23]:
classificacoes = pd.DataFrame(data = {'origem_portugues': df_portuguese['sentiment'][:500], 
                                      'classificação_hug_com_unicamp_translater': dataset['tweets_translater_unicamp'].apply(classifierD),
                                      'classificação_rabin_com_unicamp_translater': dataset['tweets_translater_unicamp'].apply(new_classifier),
                                      'classificação_hug_com_face_translater': dataset['tweets_translater_face'].apply(classifierD),
                                      'classificação_rabin_com_face_translater': dataset['tweets_translater_face'].apply(new_classifier)})

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [31]:
classifier(Tweets_in_English['tweets_translater_unicamp'][0])

[{'label': 'NEGATIVE', 'score': 0.9786376357078552}]

In [ ]:
classificacoes['classificação_hug'] = classificacoes['classificação_hug'].apply(lambda x: x[0]['label'])
classificacoes['classificação_hug_face'] = classificacoes['classificação_hug_face'].apply(lambda x: x[0]['label'])

In [ ]:
classificacoes['classificação_hug'] = classificacoes['classificação_hug'].replace('POSITIVE', 1).replace('NEGATIVE',-1)
classificacoes['classificação_hug_face'] = classificacoes['classificação_hug_face'].replace('POSITIVE', 1).replace('NEGATIVE',-1)

In [ ]:
classificacoes['classificação_rabin_hug'] = classificacoes['classificação_rabin_hug'].replace(0, -1).replace(1,0).replace(2,1)
classificacoes['classificação_rabin_face'] = classificacoes['classificação_rabin_face'].replace(0, -1).replace(1,0).replace(2,1)

In [ ]:
Tweets_in_English

In [ ]:
classificacoes

## Acuracia da origem com classificador do huggingface e traduzido com o tradutor da unicamp

In [91]:
soma = 0
for i in range (len(classificacoes)):
    if(int(classificacoes['origem_portugues'][i]) == int(classificacoes['classificação_uni'][i])):
        soma+=1

soma/len(classificacoes)

0.6

## Acuracia da origem com classificador do rabin e traduzido com o tradutor da unicamp

In [92]:
soma = 0
for i in range (len(classificacoes)):
    if(int(classificacoes['origem_portugues'][i]) == int(classificacoes['classificação_rabin_uni'][i])):
        soma+=1

soma/len(classificacoes)

0.5

## Acuracia da origem com classificador do huggingface e traduzido com o tradutor da m2m100 ou face

In [93]:
soma = 0
for i in range (len(classificacoes)):
    if(int(classificacoes['origem_portugues'][i]) == int(classificacoes['classificação_uni_face'][i])):
        soma+=1

soma/len(classificacoes)

0.55

## Acuracia da origem com classificador do rabin e traduzido com o tradutor da m2m100 ou face

In [94]:
soma = 0
for i in range (len(classificacoes)):
    if(int(classificacoes['origem_portugues'][i]) == int(classificacoes['classificação_rabin_face'][i])):
        soma+=1

soma/len(classificacoes)

0.65